In [1]:
import pandas as pd
import numpy as np
import re
import json
import time
from datetime import datetime
import string
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class InsuranceTaxonomyClassifier:
    """
    Clasificator care potrivește companii cu etichete din taxonomia de asigurări.
    Folosește tehnici NLP și potrivire bazată pe similaritate.
    """

    def __init__(self, taxonomy_path, threshold=0.15, verbose=True):
        """
        Inițializez clasificatorul cu datele taxonomiei și configurația.

        Args:
            taxonomy_path (str): Calea către fișierul CSV cu taxonomia
            threshold (float): Pragul de similaritate pentru potrivire
            verbose (bool): Dacă se afișează mesaje de progres
        """
        self.threshold = threshold
        self.verbose = verbose

        if self.verbose:
            print(f"[{self._get_timestamp()}] Inițializare clasificator...")

        # Încarc datele taxonomiei
        self.taxonomy_df = pd.read_csv(taxonomy_path)
        self.taxonomy_labels = self.taxonomy_df['label'].tolist()

        # Creez lista de stopwords personalizată
        self.stopwords = self._create_stopwords()

        # Creez mapări pentru domenii specifice
        self.domain_mappings = self._create_domain_mappings()

        # Creez dicționarul de sinonime
        self.synonyms = self._create_synonym_mappings()

        # Procesez etichetele taxonomiei
        if self.verbose:
            print(f"[{self._get_timestamp()}] Procesare {len(self.taxonomy_labels)} etichete din taxonomie...")

        self.processed_labels = self._preprocess_taxonomy()

        # Analizez structura taxonomiei
        self.taxonomy_structure = self._analyze_taxonomy_structure()

        # Inițializez vectorizatorul TF-IDF
        self.vectorizer = TfidfVectorizer(
            analyzer='word',
            min_df=1,
            max_df=0.95,
            ngram_range=(1, 3),  # Include unigrame, bigrame și trigrame
            sublinear_tf=True  # Aplicăm scală logaritmică pentru TF
        )

        # Pregătesc vectorizatorul cu textul din taxonomie
        all_taxonomy_text = [' '.join(label_info['keywords']) for label_info in self.processed_labels]
        self.vectorizer.fit(all_taxonomy_text)

        # Transform etichetele taxonomiei în vectori TF-IDF
        self.taxonomy_vectors = self.vectorizer.transform(all_taxonomy_text)

        if self.verbose:
            print(f"[{self._get_timestamp()}] Clasificator inițializat cu succes!")

    def _get_timestamp(self):
        """Returnez timestamp-ul curent formatat."""
        return datetime.now().strftime("%H:%M:%S")

    def _create_stopwords(self):
        """Creez o listă personalizată de stopwords pentru domeniul de asigurări."""
        # Stopwords de bază
        basic_stopwords = {'the', 'and', 'a', 'an', 'in', 'on', 'at', 'to', 'for', 'with', 'by', 'of', 'from', 'as',
                           'is', 'are', 'was', 'were', 'be', 'been', 'being', 'that', 'this', 'these', 'those', 'have',
                           'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'would', 'should', 'could', 'can',
                           'will'}

        # Stopwords specifice domeniului de business
        business_stopwords = {'inc', 'llc', 'ltd', 'company', 'corporation', 'corp', 'co', 'service', 'services',
                              'provider', 'providers', 'business', 'enterprise'}

        return basic_stopwords.union(business_stopwords)

    def _create_domain_mappings(self):
        """Creez mapări de cuvinte cheie specifice domeniului pentru o mai bună înțelegere a contextului."""
        return {
            'construction': ['building', 'contractor', 'engineering', 'infrastructure', 'development',
                            'project', 'construction', 'builder', 'architect', 'structural'],
            'healthcare': ['medical', 'health', 'clinic', 'hospital', 'patient', 'care', 'wellness',
                          'therapy', 'treatment', 'doctor', 'physician', 'dental', 'healthcare'],
            'agriculture': ['farm', 'crop', 'plant', 'soil', 'harvest', 'cultivation', 'field', 'livestock',
                           'agricultural', 'farming', 'grower', 'breeding', 'cattle', 'dairy'],
            'technology': ['software', 'hardware', 'it', 'computer', 'digital', 'tech', 'application',
                          'system', 'network', 'programming', 'database', 'cloud', 'cyber', 'analytics'],
            'manufacturing': ['production', 'factory', 'fabrication', 'assembly', 'industrial', 'processing',
                            'manufacturing', 'machinery', 'automation', 'quality', 'component', 'plant'],
            'retail': ['store', 'shop', 'sales', 'customer', 'merchandise', 'consumer', 'mall',
                      'retail', 'wholesale', 'ecommerce', 'seller', 'buyer', 'shopper', 'product'],
            'food': ['restaurant', 'meal', 'cuisine', 'culinary', 'catering', 'dietary', 'nutrition',
                    'food', 'beverage', 'cooking', 'dining', 'chef', 'bakery', 'grocery'],
            'transportation': ['logistics', 'shipping', 'freight', 'delivery', 'hauling', 'trucking', 'transport',
                             'fleet', 'carrier', 'transit', 'distribution', 'vehicle', 'cargo'],
            'education': ['school', 'training', 'learning', 'teaching', 'academic', 'educational', 'instruction',
                         'classroom', 'student', 'teacher', 'professor', 'curriculum', 'course'],
            'finance': ['banking', 'investment', 'financial', 'money', 'fiscal', 'economic', 'accounting',
                       'finance', 'loan', 'credit', 'mortgage', 'capital', 'asset', 'debt'],
            'legal': ['law', 'attorney', 'legal', 'counsel', 'litigation', 'judicial', 'regulatory',
                     'legislation', 'compliance', 'contract', 'lawyer', 'court', 'rights'],
            'energy': ['power', 'electricity', 'gas', 'utilities', 'fuel', 'renewable', 'energy',
                      'solar', 'wind', 'hydro', 'nuclear', 'oil', 'coal', 'generation'],
            'real_estate': ['property', 'real estate', 'leasing', 'rental', 'tenant', 'building', 'housing',
                           'commercial', 'residential', 'apartment', 'condominium', 'landlord'],
            'automotive': ['vehicle', 'car', 'auto', 'automotive', 'repair', 'mechanic', 'motor',
                          'dealer', 'garage', 'maintenance', 'parts', 'service', 'collision'],
            'design': ['designer', 'creative', 'aesthetic', 'artistic', 'layout', 'visual',
                      'graphic', 'interior', 'fashion', 'product', 'industrial', 'architecture'],
            'consulting': ['advisory', 'consultant', 'guidance', 'counsel', 'strategy', 'recommendation',
                          'consulting', 'professional', 'expertise', 'specialist', 'advisor'],
            'media': ['publishing', 'broadcast', 'film', 'production', 'content', 'entertainment',
                     'media', 'news', 'television', 'radio', 'digital', 'social', 'advertising'],
            'cosmetics': ['beauty', 'makeup', 'skincare', 'salon', 'spa', 'cosmetic',
                         'hair', 'nail', 'facial', 'treatment', 'product', 'aesthetic'],
            'sports': ['fitness', 'athletic', 'exercise', 'gym', 'sport', 'recreation',
                      'training', 'coaching', 'equipment', 'facility', 'player', 'team'],
            'maintenance': ['repair', 'upkeep', 'service', 'maintain', 'fix', 'cleaning',
                          'maintenance', 'inspection', 'restoration', 'preventative', 'replacement'],
            'animal': ['pet', 'veterinary', 'animal', 'livestock', 'fauna', 'wildlife',
                      'veterinarian', 'breeding', 'shelter', 'care', 'training', 'grooming'],
            'security': ['protection', 'guard', 'surveillance', 'safety', 'alarm', 'monitor',
                        'security', 'detection', 'prevention', 'enforcement', 'risk'],
            'insurance': ['policy', 'coverage', 'premium', 'claim', 'insured', 'insurer',
                         'underwriting', 'deductible', 'risk', 'liability', 'benefit']
        }

    def _create_synonym_mappings(self):
        """Creez un dicționar de sinonime pentru termeni din domeniul asigurărilor."""
        return {
            # Sinonime generale
            'insurance': ['coverage', 'protection', 'indemnity', 'assurance'],
            'policy': ['plan', 'contract', 'agreement', 'document'],
            'premium': ['payment', 'contribution', 'subscription', 'fee'],
            'claim': ['demand', 'request', 'application', 'petition'],
            'risk': ['hazard', 'peril', 'danger', 'exposure', 'threat'],
            'damage': ['harm', 'injury', 'impairment', 'destruction', 'loss'],

            # Sinonime pentru tipuri de asigurări
            'auto': ['car', 'vehicle', 'motor', 'automobile'],
            'home': ['house', 'property', 'dwelling', 'residence', 'building'],
            'health': ['medical', 'healthcare', 'wellness', 'wellbeing'],
            'business': ['commercial', 'enterprise', 'company', 'corporate', 'industrial'],

            # Sinonime pentru profesii din asigurări
            'agent': ['representative', 'advisor', 'consultant', 'broker'],
            'adjuster': ['examiner', 'investigator', 'assessor', 'appraiser'],
            'underwriter': ['risk assessor', 'evaluator', 'analyst'],

            # Sinonime pentru concepte de risc
            'accident': ['mishap', 'incident', 'misfortune', 'casualty'],
            'disaster': ['catastrophe', 'calamity', 'emergency', 'crisis'],
            'liability': ['responsibility', 'obligation', 'accountability', 'duty'],
            'coverage': ['protection', 'insurance', 'security', 'safeguard']
        }

    def _analyze_taxonomy_structure(self):
        """Analizez structura taxonomiei pentru a identifica posibile relații părinte-copil."""
        structure = {
            'categories': {},
            'parent_child_relations': [],
            'related_labels': {}
        }

        # Grupez etichetele în posibile categorii
        for label in self.processed_labels:
            words = label['words']
            for word in words:
                if len(word) > 4:  # Folosim doar cuvintele semnificative
                    if word not in structure['categories']:
                        structure['categories'][word] = []
                    structure['categories'][word].append(label['original'])

        # Identific posibile relații părinte-copil
        for i, label1 in enumerate(self.processed_labels):
            related_labels = []

            for j, label2 in enumerate(self.processed_labels):
                if i != j:
                    # Calculez similaritatea între etichete
                    common_words = set(label1['words']).intersection(set(label2['words']))
                    if common_words:
                        related_labels.append({
                            'label': label2['original'],
                            'common_words': list(common_words),
                            'similarity': len(common_words) / max(len(label1['words']), len(label2['words']))
                        })

                    # Verific relații părinte-copil
                    if label1['clean_label'] in label2['clean_label'] and label1['clean_label'] != label2['clean_label']:
                        structure['parent_child_relations'].append({
                            'parent': label1['original'],
                            'child': label2['original']
                        })

            # Sortez etichetele asociate după similaritate
            related_labels = sorted(related_labels, key=lambda x: x['similarity'], reverse=True)
            structure['related_labels'][label1['original']] = related_labels[:5]  # Păstrează top 5

        return structure

    def _preprocess_text(self, text):
        """Procesează textul prin eliminarea caracterelor speciale, normalizare etc."""
        if not isinstance(text, str):
            return ""

        # Convertesc la litere mici
        text = text.lower()

        # Elimin punctuația
        translator = str.maketrans('', '', string.punctuation.replace('-', ''))  # Păstrăm cratimele
        text = text.translate(translator)

        # Elimin numere și caractere speciale
        text = re.sub(r'[^\w\s-]', ' ', text)

        # Elimin cuvinte comune foarte scurte și stopwords
        words = [word for word in text.split() if len(word) > 2 and word not in self.stopwords]

        return ' '.join(words)

    def _parse_business_tags(self, tags_str):
        """Analizez tagurile de business din reprezentarea string."""
        try:
            # Înlocuiesc ghilimele simple cu duble pentru parsarea JSON
            fixed_json = tags_str.replace("'", '"')
            return json.loads(fixed_json)
        except:
            if isinstance(tags_str, str) and tags_str.startswith('[') and tags_str.endswith(']'):
                # Extrag elementele dintre paranteze și împarte după virgulă
                items = tags_str[1:-1].split(',')
                # Curăț fiecare element
                return [item.strip().strip("'").strip('"') for item in items]
            return []

    def _preprocess_taxonomy(self):
        """Procesez etichetele taxonomiei pentru a extrage caracteristici și domenii cheie."""
        processed_labels = []

        for label in self.taxonomy_labels:
            # Procesare de bază a textului
            clean_label = self._preprocess_text(label)

            # Creez variație fără sufixul "Services"
            without_services = re.sub(r'\sservices$', '', clean_label)

            # Extrag cuvinte
            words = [word for word in clean_label.split() if len(word) > 2 and word not in self.stopwords]

            # Adaug sinonime pentru cuvintele cheie
            expanded_words = words.copy()
            for word in words:
                if word in self.synonyms:
                    for synonym in self.synonyms[word]:
                        if synonym not in expanded_words:
                            expanded_words.append(synonym)

            # Identific domeniile relevante
            related_domains = []
            domain_keywords = []

            # Identific domeniile relevante
            for domain, keywords in self.domain_mappings.items():
                matches = sum(1 for keyword in keywords if keyword in clean_label)
                if matches > 0:
                    related_domains.append(domain)
                    domain_keywords.extend([kw for kw in keywords if kw not in domain_keywords])

            # Creez n-grame pentru un context mai bun
            bigrams = []
            for i in range(len(words) - 1):
                bigrams.append(f"{words[i]} {words[i+1]}")

            # Creez lista de cuvinte cheie
            keywords = list(set(expanded_words + domain_keywords))

            processed_labels.append({
                'original': label,
                'clean_label': clean_label,
                'without_services': without_services,
                'words': words,
                'expanded_words': expanded_words,
                'bigrams': bigrams,
                'related_domains': related_domains,
                'keywords': keywords + bigrams  # Combin pentru o reprezentare mai bogată
            })

        return processed_labels

    def _preprocess_company(self, company):
        """Procesez datele companiei pentru clasificare."""
        # Analizez tagurile de business
        if isinstance(company['business_tags'], str):
            business_tags = self._parse_business_tags(company['business_tags'])
        else:
            business_tags = []

        # Curăț câmpurile de text
        clean_description = self._preprocess_text(company['description'])
        clean_tags = self._preprocess_text(' '.join(business_tags))
        clean_sector = self._preprocess_text(company['sector'])
        clean_category = self._preprocess_text(company['category'])
        clean_niche = self._preprocess_text(company['niche'])

        # Creez text combinat pentru analiză (cu ponderare câmpuri)
        combined_text = (
            f"{clean_description} {clean_description} "  # Pondere dublă pentru descriere
            f"{clean_tags} {clean_tags} {clean_tags} "  # Pondere triplă pentru taguri de business
            f"{clean_sector} {clean_sector} "           # Pondere dublă pentru sector
            f"{clean_category} {clean_category} "       # Pondere dublă pentru categorie
            f"{clean_niche} {clean_niche}"              # Pondere dublă pentru nișă
        )

        # Extrag cuvinte cheie specifice domeniului
        domain_presence = {}
        for domain, keywords in self.domain_mappings.items():
            matches = sum(keyword in combined_text for keyword in keywords)
            if matches > 0:
                domain_presence[domain] = matches

        # Creez vector de document folosind TF-IDF
        company_vector = self.vectorizer.transform([combined_text])

        return {
            'original': company,
            'business_tags': business_tags,
            'clean_description': clean_description,
            'clean_tags': clean_tags,
            'clean_sector': clean_sector,
            'clean_category': clean_category,
            'clean_niche': clean_niche,
            'combined_text': combined_text,
            'domain_presence': domain_presence,
            'vector': company_vector
        }

    def _score_company_for_label(self, processed_company, label_info):
        """
        Calculez un scor de similaritate între o companie și o etichetă din taxonomie.
        """
        score = 0

        # Similaritate vectorială (similaritate cosinus TF-IDF)
        vector_similarity = cosine_similarity(
            processed_company['vector'],
            self.vectorizer.transform([' '.join(label_info['keywords'])])
        )[0][0]

        # Scalează până la procent și adaugă la scor
        score += vector_similarity * 50

        # Verific potriviri exacte
        clean_label = label_info['clean_label']
        without_services = label_info['without_services']

        # Potrivire exactă a etichetei
        if clean_label in processed_company['combined_text']:
            score += 25
        elif without_services != clean_label and without_services in processed_company['combined_text']:
            score += 20

        # Potrivirea tagurilor de business (pondere mai mare pentru potriviri exacte în taguri)
        if any(clean_label in tag.lower() for tag in processed_company['business_tags']):
            score += 20
        elif any(without_services in tag.lower() for tag in processed_company['business_tags']):
            score += 15

        # Potrivire la nivel de cuvânt
        for word in label_info['words']:
            if word in processed_company['combined_text']:
                score += 2

            # Pondere extra pentru tagurile de business care conțin termeni cheie
            if any(word in tag.lower() for tag in processed_company['business_tags']):
                score += 3

        # Potrivire bigrame (mai bună pentru context)
        for bigram in label_info['bigrams']:
            if bigram in processed_company['combined_text']:
                score += 5

        # Scorare de relevanță pentru domeniu
        for domain in label_info['related_domains']:
            if domain in processed_company['domain_presence']:
                # Potrivire de domeniu - mai puternică dacă ambele au prezență ridicată
                score += 2 * min(3, processed_company['domain_presence'][domain])

        # Scoruri pentru potrivirea câmpurilor cheie (sector, categorie, nișă)
        fields = {'sector': processed_company['clean_sector'],
                 'category': processed_company['clean_category'],
                 'niche': processed_company['clean_niche']}

        for field_name, field_value in fields.items():
            if clean_label in field_value or without_services in field_value:
                # Pondere mai mare pentru potrivirile din câmpuri structurale
                score += 8

            # Verific și potriviri la nivel de cuvânt în aceste câmpuri
            for word in label_info['words']:
                if word in field_value:
                    score += 3

        return score

    def classify_company(self, company, top_n=3):
        """
        Clasifică o companie în cele mai relevante etichete din taxonomia de asigurări.

        Args:
            company (dict): Datele companiei cu descriere, business_tags, sector, categorie, nișă
            top_n (int): Numărul de potriviri de top de returnat

        Returns:
            dict: Rezultatele clasificării cu etichete potrivite și scoruri
        """
        # Preprocesez datele companiei
        processed_company = self._preprocess_company(company)

        # Calculez scoruri pentru fiecare etichetă din taxonomie
        scores = []
        for idx, label_info in enumerate(self.processed_labels):
            score = self._score_company_for_label(processed_company, label_info)
            scores.append({
                'label': label_info['original'],
                'score': score
            })

        # Sortez după scor în ordine descrescătoare
        sorted_scores = sorted(scores, key=lambda x: x['score'], reverse=True)

        # Obțin primele N potriviri
        top_matches = sorted_scores[:top_n]

        # Filtrez potrivirile peste prag
        threshold_value = self.threshold * 100
        matches_above_threshold = [
            match for match in sorted_scores
            if match['score'] >= threshold_value
        ]

        # Dacă am prea multe potriviri, păstrez doar cele mai relevante
        if len(matches_above_threshold) > 5:
            # Limitez la maximum 5 potriviri
            matches_above_threshold = matches_above_threshold[:5]

        # Mă asigur că returnez cel puțin o etichetă (cea mai bună potrivire)
        if not matches_above_threshold and len(sorted_scores) > 0:
            matches_above_threshold = [sorted_scores[0]]

        return {
            'top_matches': top_matches,
            'matches_above_threshold': matches_above_threshold
        }

    def classify_companies(self, companies_df, batch_size=100):
        """
        Clasific mai multe companii într-un DataFrame, cu procesare în loturi pentru eficiență.

        Args:
            companies_df (DataFrame): DataFrame care conține datele companiilor
            batch_size (int): Numărul de companii procesate în fiecare lot

        Returns:
            DataFrame: DataFrame-ul original cu coloana insurance_label adăugată
        """
        # Creez o copie pentru a evita modificarea originalului
        result_df = companies_df.copy()

        # Adaug coloane pentru etichete de asigurare și scoruri de încredere
        result_df['insurance_label'] = None
        result_df['confidence_score'] = None

        # Timp total de procesare și statistici
        start_time = time.time()
        total_companies = len(result_df)

        if self.verbose:
            print(f"[{self._get_timestamp()}] Începerea clasificării pentru {total_companies} companii (în loturi de {batch_size})...")

        # Procesez companiile în loturi pentru o gestionare mai eficientă a memoriei
        batch_count = (total_companies + batch_size - 1) // batch_size  # Rotunjește în sus

        all_scores = []  # Pentru statistici

        for batch_idx in range(batch_count):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, total_companies)

            if self.verbose:
                print(f"[{self._get_timestamp()}] Procesare lot {batch_idx + 1}/{batch_count} (companiile {start_idx}-{end_idx-1})...")

            # Procesez fiecare companie din lot
            for idx in range(start_idx, end_idx):
                company_dict = result_df.iloc[idx].to_dict()
                classification = self.classify_company(company_dict)

                # Obțin etichetele potrivite și scorurile
                matched_items = classification['matches_above_threshold']
                matched_labels = [match['label'] for match in matched_items]

                # Unesc etichetele multiple cu punct și virgulă
                result_df.at[idx, 'insurance_label'] = '; '.join(matched_labels)

                # Adaug scorul de încredere (media scorurilor de top)
                if matched_items:
                    confidence_scores = [match['score'] for match in matched_items]
                    avg_confidence = sum(confidence_scores) / len(confidence_scores)
                    all_scores.extend(confidence_scores)  # Pentru statistici
                    result_df.at[idx, 'confidence_score'] = round(avg_confidence, 2)
                else:
                    result_df.at[idx, 'confidence_score'] = 0

            # Afișez progresul după fiecare lot
            if self.verbose:
                elapsed = time.time() - start_time
                rate = end_idx / elapsed if elapsed > 0 else 0
                remaining = (total_companies - end_idx) / rate if rate > 0 else 0

                print(f"[{self._get_timestamp()}] Procesate {end_idx}/{total_companies} companii " +
                      f"({rate:.1f} companii/sec, est. {remaining/60:.1f} min rămase)")

        # Statistici finale
        elapsed_time = time.time() - start_time
        avg_rate = total_companies / elapsed_time

        if self.verbose:
            print(f"[{self._get_timestamp()}] Clasificare finalizată în {elapsed_time:.1f} secunde!")
            print(f"[{self._get_timestamp()}] Viteză medie de procesare: {avg_rate:.1f} companii/secundă")

            # Calculez statistici de etichete
            unique_labels = set()
            for labels in result_df['insurance_label'].str.split('; '):
                unique_labels.update(labels)

            multi_label_count = sum(result_df['insurance_label'].str.contains(';'))

            # Calculez distribuția scorurilor
            if all_scores:
                score_min = min(all_scores)
                score_max = max(all_scores)
                score_avg = sum(all_scores) / len(all_scores)
                score_median = sorted(all_scores)[len(all_scores) // 2]
            else:
                score_min = score_max = score_avg = score_median = 0

            print(f"[{self._get_timestamp()}] Statistici finale:")
            print(f"  - Etichete unice utilizate: {len(unique_labels)}/{len(self.taxonomy_labels)} " +
                  f"({len(unique_labels)/len(self.taxonomy_labels)*100:.1f}%)")
            print(f"  - Companii cu etichete multiple: {multi_label_count} " +
                  f"({multi_label_count/total_companies*100:.1f}%)")
            print(f"  - Statistici scoruri: Min: {score_min:.1f}, Max: {score_max:.1f}, " +
                  f"Medie: {score_avg:.1f}, Mediană: {score_median:.1f}")

        return result_df

    def evaluate_performance(self, test_companies, sample_size=100):
        """
        Evalueaz performanța clasificatorului pe un eșantion de companii.

        Args:
            test_companies (DataFrame): Companiile pentru evaluare
            sample_size (int): Numărul de companii de eșantionat

        Returns:
            dict: Metrici de performanță
        """
        if self.verbose:
            print(f"[{self._get_timestamp()}] Evaluarea performanței pe un eșantion de {sample_size} companii...")

        if len(test_companies) > sample_size:
            sample = test_companies.sample(sample_size, random_state=42)
        else:
            sample = test_companies

        # Procesez eșantionul
        results = []
        start_time = time.time()

        for idx, row in sample.iterrows():
            company_dict = row.to_dict()
            classification = self.classify_company(company_dict)

            top_match = classification['top_matches'][0]
            matches_count = len(classification['matches_above_threshold'])

            results.append({
                'company_idx': idx,
                'sector': company_dict['sector'],
                'category': company_dict['category'],
                'top_match': top_match['label'],
                'top_score': top_match['score'],
                'matches_count': matches_count,
                'all_matches': [match['label'] for match in classification['matches_above_threshold']]
            })

        # Calculez metricile de performanță
        results_df = pd.DataFrame(results)

        # Distribuția etichetelor
        label_counts = results_df['top_match'].value_counts().to_dict()

        # Numărul mediu de potriviri per companie
        avg_matches = results_df['matches_count'].mean()

        # Distribuția pe sectoare
        sector_distribution = results_df.groupby('sector')['top_match'].apply(list).to_dict()

        # Analizez diversitatea potrivirilor
        all_match_labels = [label for matches in results_df['all_matches'] for label in matches]
        unique_labels_assigned = len(set(all_match_labels))
        label_coverage = unique_labels_assigned / len(self.taxonomy_labels) * 100

        elapsed_time = time.time() - start_time

        if self.verbose:
            print(f"[{self._get_timestamp()}] Evaluare finalizată în {elapsed_time:.2f} secunde")
            print(f"[{self._get_timestamp()}] Rezultate evaluare:")
            print(f"  - Număr mediu de potriviri per companie: {avg_matches:.2f}")
            print(f"  - Etichete unice atribuite: {unique_labels_assigned} ({label_coverage:.1f}% din taxonomie)")
            print(f"  - Top 5 etichete atribuite:")

            for label, count in sorted(label_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
                print(f"    - {label}: {count}")

        return {
            'label_distribution': label_counts,
            'avg_matches_per_company': avg_matches,
            'sector_distribution': sector_distribution,
            'unique_labels_assigned': unique_labels_assigned,
            'label_coverage_percentage': label_coverage,
            'sample_results': results
        }


def main():
    """
    Funcția principală pentru rularea clasificatorului pe datele de intrare.

    Procesul:
    1. Încarcă datele (taxonomie și companii)
    2. Inițializează clasificatorul
    3. Evaluează pe un eșantion
    4. Clasifică toate companiile
    5. Salvează rezultatele
    """
    # Calea către fișiere
    taxonomy_path = "insurance_taxonomy.csv"
    companies_path = "ml_insurance_challenge.csv"
    output_path = "annotated_companies2.csv"

    print("=" * 80)
    print("CLASIFICATOR DE TAXONOMIE PENTRU ASIGURĂRI")
    print("=" * 80)

    # Încarc datele
    print("\nÎncărcarea datelor...")
    start_time = time.time()
    taxonomy_df = pd.read_csv(taxonomy_path)
    companies_df = pd.read_csv(companies_path)
    data_load_time = time.time() - start_time

    print(f"Au fost încărcate {len(taxonomy_df)} etichete de taxonomie")
    print(f"Au fost încărcate {len(companies_df)} companii")
    print(f"Date încărcate în {data_load_time:.2f} secunde")

    # Inițializez clasificatorul
    print("\nInițializarea clasificatorului...")
    start_time = time.time()
    classifier = InsuranceTaxonomyClassifier(
        taxonomy_path=taxonomy_path,
        threshold=0.15,
        verbose=True
    )
    init_time = time.time() - start_time
    print(f"Clasificator inițializat în {init_time:.2f} secunde")

    # Evaluez pe un eșantion înainte de clasificarea completă
    print("\nEvaluarea performanței clasificatorului pe un eșantion...")
    evaluation = classifier.evaluate_performance(companies_df, sample_size=200)

    # Determin dimensiunea optimă a lotului în funcție de setul de date
    batch_size = min(100, max(10, len(companies_df) // 20))  # Între 10 și 100, aprox. 5% din total

    # Clasific toate companiile
    print("\nClasificarea tuturor companiilor...")
    results_df = classifier.classify_companies(companies_df, batch_size=batch_size)

    # Salveaz rezultatele
    print(f"\nSalvarea setului de date adnotat în {output_path}...")
    results_df.to_csv(output_path, index=False)

    print("\nProcesul de clasificare finalizat cu succes!")


if __name__ == "__main__":
    # Execut funcția principală
    main()

CLASIFICATOR DE TAXONOMIE PENTRU ASIGURĂRI

Încărcarea datelor...
Au fost încărcate 220 etichete de taxonomie
Au fost încărcate 9494 companii
Date încărcate în 0.06 secunde

Inițializarea clasificatorului...
[16:21:28] Inițializare clasificator...
[16:21:28] Procesare 220 etichete din taxonomie...
[16:21:28] Clasificator inițializat cu succes!
Clasificator inițializat în 0.05 secunde

Evaluarea performanței clasificatorului pe un eșantion...
[16:21:28] Evaluarea performanței pe un eșantion de 200 companii...
[16:21:59] Evaluare finalizată în 31.38 secunde
[16:21:59] Rezultate evaluare:
  - Număr mediu de potriviri per companie: 4.00
  - Etichete unice atribuite: 169 (76.8% din taxonomie)
  - Top 5 etichete atribuite:
    - Wood Product Manufacturing: 23
    - Training Services: 12
    - Arts Services: 10
    - Business Development Services: 8
    - Financial Services: 6

Clasificarea tuturor companiilor...
[16:21:59] Începerea clasificării pentru 9494 companii (în loturi de 100)...
[16